In [2]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [21]:
from pyspark.sql import SparkSession
from google.cloud import storage
from pyspark.sql.functions import col, create_map, lit, concat
from itertools import chain
import json

In [22]:
# create spark session
spark = SparkSession.builder.getOrCreate()

In [23]:
# create file links
bucket = "dataproc-staging-us-central1-123791584787-rwa2xlbh"
data_folder = "data"
client = storage.Client()
files = [blob.name for blob in client.list_blobs(bucket, prefix=data_folder)]
#files = [blob.path for blob in client.list_blobs(bucket, prefix=data_folder)]
files = ["gs://"+bucket+"/"+file for file in files]
#files

In [24]:
# read files
df_list=[]
for filename in sorted(files):
    data = spark.read.option("header",True).option("multiLine", "true").csv(filename)
    data = data.drop("description").drop("tags").drop("channel_id")
    if "BR" in filename:
        data = data.withColumn("region", lit("Brazil"))
    elif 'CA' in filename:
        data = data.withColumn("region", lit("Canada"))
    elif 'DE' in filename:
        data = data.withColumn("region", lit("Germany"))
    elif 'FR' in filename:
        data = data.withColumn("region", lit("France"))
    elif 'GB' in filename:
        data = data.withColumn("region", lit("Great Britain"))
    elif 'IN' in filename:
        data = data.withColumn("region", lit("India"))
    elif 'JP' in filename:
        data = data.withColumn("region", lit("Japan"))
    elif 'KR' in filename:
        data = data.withColumn("region", lit("Korea"))
    elif 'MX' in filename:
        data = data.withColumn("region", lit("Mexico"))
    elif 'RU' in filename:
        data = data.withColumn("region", lit("Russia"))
    elif 'US' in filename:
        data = data.withColumn("region", lit("USA")) 
    df_list.append(data)

21/11/18 08:33:31 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
21/11/18 08:33:32 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at c

In [25]:
df_list[0].show(10)

In [26]:
df_all = df_list[0]
for i in range(1,len(df_list),1):
    df_all = df_all.union(df_list[i])


In [27]:
df_all = df_all.drop("channelId").drop("channelTitle").drop("tags").drop("comments_disabled").drop("ratings_disabled").drop("description")

In [10]:
# # Run some checks
# print(df_all.count())
# print([df.count() for df in df_list])
# print(len(df_all.columns))
# print([len(df.columns) for df in df_list])

In [28]:
df_all = df_all.withColumn("video_url_prefix", lit("https://www.youtube.com/watch?v="))\
                .withColumn("video_links", concat(col("video_url_prefix"), col("video_id")))

In [29]:
df_all = df_all.drop("video_url_prefix")

In [30]:
# get the categories
categories_folder = "categories"
categories_list = [json.loads(blob.download_as_string(client=None)) for blob in client.list_blobs(bucket, prefix=categories_folder)]
categories_map = {}
for caregories in categories_list: 
    for item in caregories["items"]:
        categories_map[item["id"]] = item["snippet"]["title"]

In [31]:
#source: https://stackoverflow.com/questions/42980704/pyspark-create-new-column-with-mapping-from-a-dict
mapping_expr = create_map([lit(x) for x in chain(*categories_map.items())])
df_all =df_all.withColumn("category_name", mapping_expr.getItem(col("categoryId")))

In [32]:
df_all.show(3)

In [34]:
df_all.write.csv("gs://"+bucket+"/clean_data/")

In [38]:
data = "gs://"+bucket+"/clean_data/"

In [41]:
dataframe = spark.read.option("header",True).option("multiLine", "true").csv(data)

21/11/18 08:46:59 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
21/11/18 08:46:59 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at c

In [42]:
dataframe.show()

+-----------+----------------------------------------------+--------------------+---+--------------------+-------+------+-----+-----+----------------------------------------------+------+-------------------------------------------+----------------+
|s9FH4rDMvds|LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!|2020-08-11T22:21:49Z| 22|2020-08-12T00:00:00Z| 263835| 85095|  487| 4500|https://i.ytimg.com/vi/s9FH4rDMvds/default.jpg|Brazil|https://www.youtube.com/watch?v=s9FH4rDMvds|  People & Blogs|
+-----------+----------------------------------------------+--------------------+---+--------------------+-------+------+-----+-----+----------------------------------------------+------+-------------------------------------------+----------------+
|jbGRowa5tIk|                          ITZY “Not Shy” M/...|2020-08-11T15:00:13Z| 10|2020-08-12T00:00:00Z|6000070|714310|15176|31040|                          https://i.ytimg.c...|Brazil|                       https://www.youtu...|           Music|
|3Ef